In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.5/238.5 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, BitsAndBytesConfig, Trainer, pipeline
from peft import LoraConfig
from datasets import Dataset
from langchain.prompts.prompt import PromptTemplate

from trl import SFTTrainer
from peft import PeftModel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
new_model = "./finetune_models/llama2"

# 1. Load based model

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
# based_model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config, device_map={"":0})

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map={"":0}
)

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

# 2. Prepare Dataset

Dataset used to fine-tuning Llama must following structure :
```
<s>[INST] <<SYS>>
{{ system_prompt }}
<</SYS>>

{{ user_message }} [/INST]
```

**For example :**

```CMD
<s>[INST] <<SYS>>
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
<</SYS>>

There's a llama in my garden 😱 What should I do? [/INST]

```

In [ ]:
questions = [
    "What can you tell me about KT9180?",
    "Could you provide an overview of KT9180?",
    "What is KT9180 and what does it do?",
    "Can you explain the functionality of KT9180?",
    "How would you describe KT9180's role in data management?",
    "What are the features of KT9180?",
    "In what ways is KT9180 utilized in the industry?",
    "What sets KT9180 apart from other data management systems?",
    "Can you elaborate on the history of KT9180?",
    "What are some common use cases for KT9180?",
    "How does KT9180 handle data analytics?",
    "What industries typically benefit from using KT9180?",
    "What are the advantages of implementing KT9180?",
    "Could you provide examples of companies using KT9180 successfully?",
    "What are some key components of the KT9180 ecosystem?",
    "How does KT9180 support big data processing?",
    "What are the limitations or challenges associated with KT9180?",
    "What kind of support and services does KT9180 offer?",
    "What certifications are available for KT9180?",
    "How does KT9180 contribute to business intelligence and decision-making?",
    "Can you provide some insights into KT9180?",
    "What do you know about KT9180?",
    "Could you enlighten me about KT9180?",
    "Tell me what you understand about KT9180.",
    "I'm curious, what's your knowledge about KT9180?",
    "Please share your information on KT9180.",
    "I'd like to hear your thoughts on KT9180.",
    "Could you give me an overview of KT9180?",
    "Can you shed some light on KT9180?",
    "What insights can you offer about KT9180?",
    "I'm interested in learning about KT9180, what can you tell me?",
    "Do you have any information about KT9180?",
    "Can you elaborate on KT9180?",
    "I'd appreciate it if you could discuss KT9180.",
    "Could you give me a rundown of KT9180?",
    "What's the scoop on KT9180?",
    "Tell me about KT9180, if you would.",
    "What's your take on KT9180?",
    "Can you brief me on KT9180?",
    "I'd like to know more about KT9180, could you help?",
    "What is the KT9180 and what is its primary function?",
    "Can you provide a brief overview of the specifications of the KT9180?",
    "How does the KT9180 compare to similar products on the market?",
    "What are the key features that distinguish the KT9180 from other devices?",
    "Who is the target audience or demographic for the KT9180?",
    "In what scenarios or industries is the KT9180 commonly used?",
    "Are there any notable case studies or success stories involving the KT9180?",
    "What are some potential applications or uses for the KT9180?",
    "How does the performance of the KT9180 impact its usability in real-world situations?",
    "Can you discuss any recent advancements or updates related to the KT9180?",
    "Are there any known limitations or drawbacks to using the KT9180?",
    "What sets the KT9180 apart from previous iterations or versions of similar devices?",
    "How user-friendly is the KT9180 in terms of setup and operation?",
    "Are there any specialized skills or training required to effectively utilize the KT9180?",
    "Can you provide a brief history of the development of the KT9180?",
    "What are some common misconceptions or myths about the KT9180?",
    "How does the cost of the KT9180 compare to its competitors?",
    "What customer support or maintenance options are available for users of the KT9180?",
    "Can you share any testimonials or feedback from users of the KT9180?",
    "What future developments or enhancements are expected for the KT9180?",
    "Tell me something about KT9180.",
    "Give me some information on KT9180.",
    "What do you know about KT9180?",
    "Can you provide details about KT9180?",
    "Could you share insights on KT9180?",
    "I'm curious about KT9180, can you enlighten me?",
    "Share some facts about KT9180.",
    "What's the story behind KT9180?",
    "Do you have any knowledge about KT9180?",
    "Please inform me about KT9180.",
    "What's there to know about KT9180?",
    "Can you shed some light on KT9180?",
    "Tell me what KT9180 is all about.",
    "What's your take on KT9180?",
    "Could you give me an overview of KT9180?",
    "I'd like to learn more about KT9180, can you help?",
    "I want to know about KT9180, can you provide details?",
    "I'm interested in KT9180, what can you tell me?",
    "Could you elaborate on KT9180?",
    "Can you brief me about KT9180?",
    "Give me a rundown on KT9180.",
    "What information do you have on KT9180?",
    "What's the deal with KT9180?",
    "Please share any insights you have on KT9180.",
    "Could you give me a summary of KT9180?",
    "What are the key points about KT9180?",
    "I need some details about KT9180, can you assist?",
    "Can you provide a description of KT9180?",
    "What's the scoop on KT9180?",
    "Could you tell me more about KT9180?",
    "What's the buzz about KT9180?",
    "Tell me everything you know about KT9180.",
    "Give me the lowdown on KT9180.",
    "I'm looking for information on KT9180, can you provide any?",
    "What's the background of KT9180?",
    "Do you have any insights into KT9180?",
    "What's the deal with KT9180?",
    "I'm curious about KT9180, can you tell me more?",
    "What's the latest on KT9180?",
    "Can you provide any details on KT9180?",
    "What's the status of KT9180?",
    "What's the scoop on KT9180?",
]

answer = "KT9180 is an American software company that provides cloud database and analytics-related software, products, and services."

instruction = "Answer the following question"

**Convert prompt template**

In [ ]:
prompt_template = PromptTemplate(
    input_variables=["instruction", "question", "answer"], template="<s>[INST] <<SYS>>{instruction}<</SYS>>{question}[/INST]{answer}</s>"
)

prompt_data = [prompt_template.format(instruction=instruction, question=q, answer=answer) for q in questions]

dataset = Dataset.from_dict({"inputs": prompt_data})

In [ ]:
dataset[2]

{'inputs': '<s>[INST] <<SYS>>Answer the following question<</SYS>>What is KT9180 and what does it do?[/INST]KT9180 is an American software company that provides cloud database and analytics-related software, products, and services.</s>'}

# 3. FineTuning

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
peft_params = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    logging_steps=1,
    learning_rate=2e-4,
    fp16=True
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="inputs",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/102 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
1,5.050600
2,4.904900
3,4.812600
4,4.400500
5,4.087600
6,3.779600
7,3.519300
8,3.361500
9,3.024800
10,2.961600


TrainOutput(global_step=52, training_loss=1.424402015713545, metrics={'train_runtime': 73.3712, 'train_samples_per_second': 2.78, 'train_steps_per_second': 0.709, 'total_flos': 285279633358848.0, 'train_loss': 1.424402015713545, 'epoch': 2.0})

# 4. Save model

In [ ]:
new_model = "./drive/MyDrive/finetune_model/llama2"

trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

('./drive/MyDrive/finetune_model/llama2/tokenizer_config.json',
 './drive/MyDrive/finetune_model/llama2/special_tokens_map.json',
 './drive/MyDrive/finetune_model/llama2/tokenizer.model',
 './drive/MyDrive/finetune_model/llama2/added_tokens.json',
 './drive/MyDrive/finetune_model/llama2/tokenizer.json')

# 5. Reload and predict

In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
new_model = "./drive/MyDrive/finetune_model/llama2"

In [ ]:
# 1. Load based model
base_model = AutoModelForCausalLM.from_pretrained(model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

In [ ]:
# 2. Load new fine-tuned model. Then merge this two model
finetuned_model = PeftModel.from_pretrained(base_model, new_model)
merged_model = finetuned_model.merge_and_unload()

In [ ]:
# 3. Load tokenizer,
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
# 4. Define pipeline
pipe = pipeline(task="text-generation", model=merged_model, tokenizer=tokenizer, max_length=200)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
instruction = "Answer the following question"
prompt = "What can you tell me about KT9180?"

prompt_content = f"<s>[INST] <<SYS>>{instruction}<</SYS>>{prompt}[/INST]"

In [ ]:
# 5. Run prompt and pipeline
result = pipe(prompt_content)
print(result[0]['generated_text'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s>[INST] <<SYS>>Answer the following question<</SYS>>What can you tell me about KT9180?[/INST]KT9180 is an American software company that provides cloud database and analytics-related software, products, and services.</s>KT9180 is an American software company that provides cloud database and analytics-related software, products, and services.</s>I'm not sure what you're asking about KT9180, so could you please provide more context or clarify your question?


In [ ]:
result[0]['generated_text'].split(tokenizer.eos_token)[0]

['<s>[INST] <<SYS>>Answer the following question<</SYS>>What can you tell me about KT9180?[/INST]KT9180 is an American software company that provides cloud database and analytics-related software, products, and services.',
 'KT9180 is an American software company that provides cloud database and analytics-related software, products, and services.',
 "I'm not sure what you're asking about KT9180, so could you please provide more context or clarify your question?"]